In [1]:
import pandas as pd
import numpy as np
from qdrant_client import models, QdrantClient
# from sentence_transformers import SentenceTransformer
from huggingface_hub import login
from tqdm import tqdm

ModuleNotFoundError: No module named 'qdrant_client'

In [2]:
import os

In [3]:
huggingface_token = os.environ.get("HUGGINGFACE_TOKEN")

In [4]:
login(token=huggingface_token)

In [5]:
df = pd.read_csv('podcast_data.csv')

In [7]:
df.head()

,episode,title,summary,song_recommendation
0,1,沒錄到後半部的NMSL前世今生,NMSL你媽死了 最早是在CS看到的 msl是一個丹麥二線玩家（通常是一線貼紙比較貴）一個團...,NaN
1,2,我實在是想不到比多人運動更委婉的說法,A 片打炮算偷情 戴 VR 做愛舉報通姦罪 做公投 究極 no 選擇 用 VR 跟老婆做愛 ...,NaN
2,3,你聽過我爸連戰嗎?,強烈要求清大人社把學費還給艾莉莎莎，宿舍費不退 友人爭論時很愛拿綽號倒過來念來 不好笑的人都...,NaN
3,4,抄襲仔求生指南與周遊記挑戰失敗,Lofi house 便當怎麼抄襲都沒差 破鍋配爛蓋 不流汗的室內設計 風格師 5566 抄...,NaN
4,5,傑出人士表揚，李來希先生。,podcast 比 yt 好賺的原因 頻率 光頭葛格 抖內30元 祝生日快樂 先射箭再畫靶 ...,NaN


In [6]:
from fastembed import TextEmbedding

In [7]:
model = TextEmbedding(model_name="BAAI/bge-base-en-v1.5")
print("The model BAAI/bge-small-en-v1.5 is ready to use.")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model_optimized.onnx:   0%|          | 0.00/218M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

The model BAAI/bge-small-en-v1.5 is ready to use.


In [10]:
def dataframe_to_text_list(df):
    """
    Convert a pandas DataFrame to a list of formatted strings.
    Each string contains column:value pairs separated by commas.
    
    Example: ["column1:value1,column2:value2,...", "column1:value1,column2:value2,...", ...]
    """
    text_list = []
    
    for _, row in df.iterrows():
        # Create a list of "column:value" pairs for each row
        pairs = [f"{col}:{val}" for col, val in row.items()]
        
        # Join these pairs with commas
        row_text = ";".join(pairs)
        
        # Add the formatted row to our list
        text_list.append(row_text)
    
    return text_list

In [11]:
documents = dataframe_to_text_list(df)

In [12]:
from tqdm.notebook import tqdm
import numpy as np

def embed_documents_with_progress(model, documents):
    """
    Embed documents with a progress bar using tqdm
    
    Args:
        model: The embedding model
        documents: List of documents to embed
        
    Returns:
        List of embeddings
    """
    embeddings = []
    
    # Use tqdm to create a progress bar
    for doc in tqdm(documents, desc="Embedding documents", unit="doc"):
        embedding = list(model.embed(doc))
        embeddings.append(embedding)
    
    return embeddings

In [13]:
embeddings = embed_documents_with_progress(model, documents)

Embedding documents:   0%|          | 0/444 [00:00<?, ?doc/s]

In [10]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333")

In [11]:
client.create_collection(
    collection_name="test_collection",
    vectors_config=models.VectorParams(size=768, distance=models.Distance.COSINE),
)

UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `test_collection` already exists!"},"time":0.000226222}'

In [29]:
def dataframe_to_points(model, df):
    points = []

    for _, row in tqdm(df.iterrows(), desc="Embedding rows of DataFrame"):
        # Create a list of "column:value" pairs for each row
        id_value = row['episode']
        
        pairs = [f"{col}:{val}" for col, val in row.items() if col != 'episode']
        
        # Join these pairs with commas
        row_text = ";".join(pairs)

        embedding = list(model.embed(row_text))[0]

        points.append(
            models.PointStruct(
                id=id_value,
                vector=embedding
            )
        )

    return points

In [30]:
embed_points = dataframe_to_points(model, df)

Embedding rows of DataFrame: 444it [02:08,  3.46it/s]


In [32]:
client.upsert(
    collection_name="test_collection",
    points=embed_points
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [67]:
question = ["約會必勝，告白失敗"]
que_vector = list(model.embed(question))[0]

In [68]:
search_result = client.query_points(
    collection_name="test_collection",
    query=que_vector,
    limit=10
).points

print(search_result)

[ScoredPoint(id=258, version=1, score=0.79606247, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=383, version=1, score=0.7905643, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=38, version=1, score=0.78805125, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=424, version=1, score=0.78662246, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=233, version=1, score=0.7844709, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=40, version=1, score=0.7765012, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=421, version=1, score=0.77574706, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=83, version=1, score=0.7739875, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=68, version=1, score=0.77265084, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=111, versio

In [69]:
result_episode = []
for result in search_result:
    result_episode.append(result.id)

In [70]:
df[df['episode'].isin(result_episode)]

,episode,title,summary,song_recommendation
37,38,長期徵求約會必勝奇招,談國民黨齒輪卡到\n國民黨尊重發言 先進的想法-國民黨沒有言論審查\n約會五分就愛愛 \...,淺堤 Shallow Levée -【永和 + 薄冰上跳舞 + 陷眠】Live Sessio...
39,40,躲在車子裡算甚麼英雄好漢,想控制不受控的人 只有自己不受控\n逞誠覺得 川普都做有賺的才生氣 呱吉才會好氣討罵\n呱...,NaN
67,68,禮物重量請低於十公斤,何ㄟ英文開場\n小有名氣之後 出門不能太邋遢\n6:15這集業配沒給何ㄟ錢\n7:39 交換...,NaN
82,83,交換禮物是吊床,為了大眾 當閃哥\n猴王的由來從哪來 因為根本就沒有由來 事務所的豪哥也來問是哪集\n解釋哪...,NaN
110,111,畢業典禮,泡麵 雙響泡 阿Ｑ桶麵加統一麵 何ㄟ固定維力炸醬麵煮湯麵但怕被逞誠罵（法白楊貴智也會）自詡孤...,NaN
232,233,青春審查員,任天堂是很獨裁的公司，不聽話的廠商會故意被排在和大遊戲一起發。\n最近拍的MV，打一打見血，...,NaN
257,258,告白先不要,誠誠建議聖誕節跨年都不要告白 何A有節日告白過 誠誠看蔡志忠的莊子 家倫看老子 \n誠誠自比...,NaN
382,383,你要怎麼說他不是主角,很廢的轉生異世界主角 關鍵字／句的角色是配角\n三色豆是配角 蒸蛋跟控肉？白飯到底是男一還...,許茹芸-欲哭無淚
420,421,要是有便利大裹巾就好,何a買鹽酥雞必有雞皮與甜不辣，討厭軟雞皮。\n--業配線--\n最近網路傳一張圖片房間堆滿模...,家倫: 小虎隊-我的舞伴名字叫做風，收錄在小虎隊「紅蜻蜓」。一個社恐中二覺得自己很帥。
423,424,全方位超男子 ft. 美麗本人,家倫機長再度登場\n---業配線---\n大家好，歡迎來到台灣通勤第一品牌，我是李毅誠，我是...,美麗本人推兩首：\n\nChromeo:《Fancy Footwork》\n盧廣仲 ft.美...


In [65]:
print(df[df['episode']==17].summary.values)

['端午節立蛋、哥倫布的雞蛋\n國中就抽菸的人超壞 跟寫幹署名一樣\n老鄧 純血外省人 放一堆石頭 在蘇蘇的背包\n6:30 踢狗事件 腳剌狗 沒何權協會  狂踹何ㄟ都沒人理 正義魔人（SJW）\n11:20 四大寬容 1. 來都來了2. 人都死了 3. 大過年的 4. 他只是個孩子 \n最過分的霸凌是老師起頭\n逞誠ㄒ曾經想當老師\n恐龍家長\n反霸凌小卡\n最可怕的就是 以前吃過的苦現在也要吃\n點菸給大便蹲加酒 穿錯服裝的天兵 穿內褲\n軍中恐怖故事 長官檢查不要進去\n希望全民都有打靶經驗 逞誠壓著新兵教打靶小撇步\n50:00 何ㄟ打過火箭彈\n52:00 最後一波夏日最後的回憶 折扣碼 wbliao\n究極ㄋㄡ選擇 同梯\n基隆滷肉飯 白胡椒加醋\n逞誠濃湯 哥倫布的配方\n伸港鄉的炸醬麵\n豆干、紅油炒手美景 士林大道附近\n糞中之糞 糞糞\n下班講阿迪歐斯']


In [66]:
print(df[df['episode']==38].summary.values)

['談國民黨齒輪卡到\n國民黨尊重發言   先進的想法-國民黨沒有言論審查\n約會五分就愛愛 \n你是不是只喜歡跟我愛愛 95％男生有被問過\n家倫被問是不是對身體沒興趣\n提點年輕男性 不要那麼興幹\n少年pi電影事件 自我的哲學時間\n特斯拉還沒到高點（結果節目播出不久跌了）\n修煉愛情的心酸\n新的不想給別人知道的歌 李聖傑眼底星空 任賢齊我是一隻魚\n植入挑歌晶片到家倫的腦袋裡 \n家倫妹喜歡徐懷鈺 錄音帶被家倫藏起來  家倫爸聽費玉清 張雨生（家倫也喜歡）\n逞誠談上呱吉直播放台語歌 什麼是南部口味的台語歌 非慣聽的人會想是好不好聽\n酒是舞伴你是生命，因為帶入生活感很深 所以變好聽了\n只會留下開心的記憶 逞誠 忘記何ㄟ摔車記憶 (如膠似漆捏)\n店裡有一道是辣的 忘記怎麼寫 寫中間一點 寫辣小撇步\n巳已己  已有長牙齒\n親 怎麼寫 辣怎麼寫 家倫讓手想要的樣子寫出來 肌肉記憶\n延長線插頭規格 檯燈開關不統一\n乾爹詢問要不要收納生活大改造  家倫舉例就像找他造型大改造\n約會 逞誠 攻擊型 家倫防守型\n阿英幫逞誠整理書 害他作業找不到沒帶去學校\n家倫東西都有固定的位子\n逞誠整理術 發現邏輯只有收納的日期  不是重要非重要的邏輯\n家倫整理術 存摺不見就是在逞誠的重要袋子裡 \n逞誠口直心快  先嗆\n家倫約會必殺奇招 按摩很厲害 逞誠必殺劣勢在podcast用完了  \n笑到脖子痛\n修煉愛情 阿飛與小蝴蝶 成全 癡心絕對 眼底星空 任賢齊 我是一隻魚許茹芸 張惠妹  萬芳']
